<a href="https://colab.research.google.com/github/sril1485/CE888/blob/master/Ass_2_final_house_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
os.environ['KAGGLE_USERNAME'] = "srinidhikarthick" # username from the json file
os.environ['KAGGLE_KEY'] = "15467a092669b85df36db57586c2dee5" # key from the json file
!kaggle competitions download -c sberbank-russian-housing-market

  0% 0.00/18.6k [00:00<?, ?B/s]
100% 18.6k/18.6k [00:00<00:00, 7.20MB/s]
100% 4.46M/4.46M [00:00<00:00, 17.3MB/s]

  0% 0.00/25.4k [00:00<?, ?B/s]
100% 25.4k/25.4k [00:00<00:00, 25.3MB/s]
 29% 5.00M/17.1M [00:00<00:01, 11.0MB/s]
100% 17.1M/17.1M [00:00<00:00, 31.6MB/s]
  0% 0.00/66.5k [00:00<?, ?B/s]
100% 66.5k/66.5k [00:00<00:00, 68.5MB/s]


In [0]:
!unzip \*.zip

Archive:  test.csv.zip
  inflating: test.csv                
   creating: __MACOSX/
  inflating: __MACOSX/._test.csv     

Archive:  macro.csv.zip
  inflating: macro.csv               

Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   

Archive:  train.csv.zip
  inflating: train.csv               
  inflating: __MACOSX/._train.csv    

4 archives were successfully processed.


In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from tqdm import tqdm_notebook
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.ensemble import GradientBoostingRegressor as GBR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import  make_scorer
from sklearn.metrics import mean_squared_error

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
train = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")

In [0]:
import numpy as np
## missing values for train data
for i in train.columns:
    if train[i].dtype == 'object':
      train[i] = train[i].fillna(train[i].mode().iloc[0])
    if (train[i].dtype == 'int' or train[i].dtype == 'float'):
      train[i] = train[i].fillna(np.mean(train[i]))
#missing values for test data
for i in test.columns:
    if test[i].dtype == 'object':
      test[i] = test[i].fillna(test[i].mode().iloc[0])
    if (test[i].dtype == 'int' or test[i].dtype == 'float'):
      test[i] = test[i].fillna(np.mean(test[i]))



In [0]:
# replacing the categorical values into numerical values
lenc = LabelEncoder()
for i in train.columns:
    if (train[i].dtype == 'object'):
      train[i] = lenc.fit_transform(train[i].astype('str'))
      train[i] = train[i].astype('object')

for i in test.columns:
    if (test[i].dtype == 'object'):
      test[i] = lenc.fit_transform(test[i].astype('str'))
      test[i] = test[i].astype('object')

In [0]:
print("Nan values in Train set is " , train.isnull().sum().sum())
print("Nan values in Test set is " , test.isnull().sum().sum())

Nan values in Train set is  0
Nan values in Test set is  0


In [0]:
## taking sample from training data
train = train.sample(7000, random_state=12)

In [0]:
# create dummy dataframes for covarite shift 
c_train = train.drop(columns=['id','price_doc'])
c_test = test.drop(columns=['id'])

In [0]:

def findFeatures(train, test, cv = 10, thresh = 0.8, n_samples = None):
    
    #labels for train and test set
    train['set'] = 0
    test['set'] = 1

    ## Consider taking smaller sample in case of huge datasets
    if n_samples is not None:
        print("Taking {} random samples from the data".format(n_samples))
        train = train.sample(n_samples, random_state=12)
        test = test.sample(n_samples, random_state=12)


    combinedData = train.append(test)
    X = combinedData.drop(columns=['set'], axis=1)
    y = combinedData['set']

    model = RandomForestClassifier(n_estimators = 50, max_depth = 5, min_samples_leaf = 5)
    features = []

    for feature in tqdm_notebook(X.columns):
        score = cross_val_score(model, pd.DataFrame(X[feature]), y, cv=cv, scoring='roc_auc')
        if np.mean(score) > thresh:
            CS_features.append(feature)
            print(feature,np.mean(score))

    return features

In [0]:
cs_features = findFeatures(c_train, c_test, cv = 10, thresh = 0.8,)

timestamp 0.7815863007196813
life_sq 0.8369653262832225
build_year 0.7963630423322751
kitch_sq 0.8939341962635321
state 0.7565409463086368
preschool_quota 0.7323690675169854
school_quota 0.7384121730152842
hospital_beds_raion 0.8848359380584897
raion_build_count_with_material_info 0.7031376253630868
build_count_brick 0.7057844524484675
build_count_panel 0.7002048815679609
raion_build_count_with_builddate_info 0.703205955813448
build_count_1946-1970 0.7043913041311036
cafe_sum_500_min_price_avg 0.8482666518992359
cafe_sum_500_max_price_avg 0.8478856335835692
cafe_avg_price_500 0.8479298253633786
cafe_sum_1000_min_price_avg 0.7031805430264739
cafe_avg_price_1000 0.7036729858626571



In [0]:
from sklearn.model_selection import StratifiedKFold as SKF
from sklearn.metrics import roc_auc_score as AUC
from sklearn.svm import SVC

def getWeight(train, test):
    #labels for train and test set
    train['set'] = 0
    test['set'] = 1

    combinedData = train.append(test)
    X = combinedData.drop(columns=['set'], axis=1)
    y = combinedData['set']

    clf = RandomForestClassifier()
    # clf = SVC(probability=True)
    predictions = np.zeros(y.shape)
    skf = SKF(n_splits=10, shuffle=True, random_state=888)

    for kfold, (train_idx, test_idx) in enumerate(skf.split(X, y)):
        print('Training discriminator model for fold {}'.format(kfold))
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
            
        clf.fit(X_train, y_train)
        probs = clf.predict_proba(X_test)[:, 1]
        predictions[test_idx] = probs

    all_weights = np.exp(predictions)*len(train)/len(test)
    tr_weights = all_weights[:len(train)]

    return tr_weights


In [0]:
train_weights = getWeight(c_train, c_test)

Training discriminator model for fold 0
Training discriminator model for fold 1
Training discriminator model for fold 2
Training discriminator model for fold 3
Training discriminator model for fold 4
Training discriminator model for fold 5
Training discriminator model for fold 6
Training discriminator model for fold 7
Training discriminator model for fold 8
Training discriminator model for fold 9


In [0]:
y = train['price_doc']
X = train.drop(['price_doc'], axis=1)

In [0]:
# Build prediction Model without weight
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, accuracy_score

# main Model
clf = RandomForestClassifier(n_jobs=-1,max_depth=5)
clf.fit(X,y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [0]:
predictions_no_weight = clf.predict(test)

In [0]:
# Build prediction Model with weight
# main Model
clf = RandomForestClassifier(n_jobs=-1,max_depth=5)
clf.fit(X,y, sample_weight=train_weights)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [0]:
predictions_with_weight = clf.predict(test)

In [0]:
sub_df = pd.DataFrame({'id':test['id']})

In [0]:
sub_df['price_doc'] = predictions_no_weight

In [0]:
sub_df.to_csv("submission_no_weight.csv", index=False)


In [0]:
!kaggle competitions submit -c sberbank-russian-housing-market -f /content/submission_no_weight.csv -m "1st submission without CS"

100% 105k/105k [00:08<00:00, 13.3kB/s]
Successfully submitted to Sberbank Russian Housing Market

In [0]:
sub_df = pd.DataFrame({'id':test['id']})
sub_df['price_doc'] = predictions_with_weight
sub_df.to_csv("submission_with_weight.csv", index=False)

In [0]:
!kaggle competitions submit -c sberbank-russian-housing-market -f /content/submission_with_weight.csv -m "2nd submission without CS"

100% 105k/105k [00:10<00:00, 10.6kB/s]
Successfully submitted to Sberbank Russian Housing Market

In [0]:
!kaggle competitions submissions -c sberbank-russian-housing-market

fileName                    date                 description                status    publicScore  privateScore  
--------------------------  -------------------  -------------------------  --------  -----------  ------------  
submission_with_weight.csv  2020-04-21 08:51:28  2nd submission without CS  complete  1.37904      1.37471       
submission_no_weight.csv    2020-04-21 08:50:56  1st submission without CS  complete  1.38895      1.39612       
submission_no_cs.csv        2020-04-21 08:12:02  1st submission without CS  error     None         None          
